(data-processing)=
# Data Processing

## Overview

Echopype data processing funcionalities are structured into different subpackages with a series of [data processing levels](processing-levels) in mind. Once the data is converted from the raw instrument data files to standardized [`EchoData` objects](data-format:echodata-object) (or stored in `.zarr` or `.nc` format) and calibrated, the core input and output of most subsequent functions are standard [xarray `Dataset` objects](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html). This way, additonal processing functions can be easily added without needing to understand specialized objects.

## Summary of subpacakges

Echopype currently contains the following data processing subpacakges:
- [`calibrate`](echopype.calibrate): Transform raw instrument data into physicaly meaningful quantities, such as the volume backscattering strength (Sv)
  - Narrowband Sv:
    AZFP, EK60, and EK80 narrowband ("CW") mode transmission
  - Broadband Sv average over the signal frequency band: EK80 broadband ("FM") mode transmission  
- [`consolidate`](echopype.consolidate): Add additional information such as the geospatial locations, split-beam angle, etc to calibrated datasets. The source of information include:
  - Data already stored in the `EchoData` object, and hence already stored in the raw instrument data files
  - Quantities that can be computed from data stored in the `EchoData` object, e.g., for EK80 broadband data, split-beam angle can be computed from the complex samples
  - External dataset, e.g., AZFP echosounders do not record geospatial data, but typically there is a companion dataset with GPS information
- [`clean`](echopype.clean): Reduce variabilities in backscatter data by perform noise removal operations. Currently contains only a simple noise removal function implementing the algorithm in {cite}`DeRobertis2007_noise`.
- [`commongrid`](echopype.commongrid): Enhance the spatial and temporal coherence of data. Currently contain functions to compute mean volume backscattering strength (MVBS) and nautical areal backscattering coefficients (NASC) that both result in gridded data at uniform spatial and temporal intervals.
- [`qc`](echopype.qc): Handle unexpected irregularities in the data. Currently contains only functions to handle timestamp reversal in EK60/EK80 raw data.
- [`mask`](echopype.mask): Create or apply mask to segment data
- [`metrics`](echopype.metrics): Calculate simple summary statistics from data

<!-- - Compute mean volume backscattering strength (MVBS) based on:
  - ping time interval and range interval specified in physics units (seconds and meters, respectively) ([echopype.commongrid.compute_MVBS](echopype.commongrid.compute_MVBS))
  - the number of pings and sample intervals (the `ping_time` and `range_sample` dimension, respectively) ([echopype.commongrid.compute_MVBS_index_binning](echopype.commongrid.compute_MVBS_index_binning)) -->

:::{Note}
The ``clean`` and ``commongrid`` subpackages were introduced in version 0.7.0.
They contain functions previously found in the deprecated ``preprocess`` subpackage;
``preprocess`` was removed in version 0.8.0.
:::

In [ ]:
import echopype as ep

The use of these processing functions are summarized below:

#### Calibration

In [ ]:
import echopype as ep
nc_path = './converted_files/file.nc'  # path to a converted nc file
echodata = ep.open_converted(nc_path)  # create an EchoData object
ds_Sv = ep.calibrate.compute_Sv(echodata)  # obtain a dataset containing Sv, echo_range, and
                                           # the calibration and environmental parameters


:::{Note}
The calibration functions can require different input arguments depending on the echosounder (`sonar_model`). See [](data-processing:calibration) for detail.
:::

#### Nosie removal

In [ ]:
# Remove noise
ds_Sv_clean = ep.clean.remove_noise(    # obtain a denoised Sv dataset
    ds_Sv,             # calibrated Sv dataset
    range_sample_num=30,  # number of samples along the range_sample dimension for estimating noise
    ping_num=5,        # number of pings for estimating noise
)

#### Reduce data by computing MVBS

In [ ]:
# Reduce data based on physical units
ds_MVBS = ep.commongrid.compute_MVBS(
    ds_Sv,               # calibrated Sv dataset
    range_meter_bin=20,  # bin size to average along echo_range in meters
    ping_time_bin='20S'  # bin size to average along ping_time in seconds
)

# Reduce data based on sample number
ds_MVBS = ep.commongrid.compute_MVBS_index_binning(
    ds_Sv,             # calibrated Sv dataset
    range_sample_num=30,  # number of sample bins to average along the range_sample dimensionm
    ping_num=5         # number of pings to average
)


#### Saving results

Typically echopype functions do not save the calculation resuls to disk, but the returned xarray `Dataset` can be saved using native xarray method such as `to_netcdf` and `to_zarr`. For the `EchoData` object we have implemented identical methods to allow saving raw converted data to disk. For example:

In [ ]:

ed = ep.open_raw(...)
ed.to_zarr("RAW_CONVERTED_FILENAME.zarr")  # save data to Zarr format

# Some processing functions that results in an xarray Dataset ds_Sv
ds_Sv.to_netcdf("PROCESSED_FILENAME.nc")  # save data to netCDF format

(data-processing:calibration)=
## Parameter considerations for calibrating data

Calibration transforms raw data collected by the instrument into physically meaningful units by incorporating:
- Calibration parameters (`cal_params`) instrinsic to the instrument and its settings
- Environmental parameters (`env_params`), such as sound speed and absorption coefficient due to the physical environment

Echopype also requires correct input argument combinations to calibrate data, due to intrinsic differences of the echosounders.

These considerations are explained below.

### Input arguments

#### AZFP and EK60 data
For data from the AZFP and EK60 echosounders, since these instruments can only transmit narrowband data, you do not have to specify any additional argument when calibrating data, and the following should always work:
```python
ed = ep.open_raw("PATH_TO_RAW_FILE", sonar_model="EK60")
ds_Sv = ep.calibrate.compute_Sv(ed)
```

#### EK80 data
For data from the EK80 echosounder, since both narrowband and broadband transmissions are possible for different channels, and for narrowband transmissions the data can be stored in two forms, you need to specify argument combinations corresponding to the data you want to calibrate.

For computing band-averaged Sv from broadband EK80 data, use
```python
ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode="BB", encode_mode="complex")
```
Here, `waveform_mode="BB"` indicates that the data you want to calibrate are the channels set to do broadband transmissions. `encode_mode="complex"` indicates that these data are stored as complex samples. The function will raise an error if there are no broadband data found in the provided `EchoData` object (`ed`).

For computing narrowband Sv from narrowband EK80 data:
- If the data is stored as complex samples, use
    ```
    ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode="CW", encode_mode="complex")
    ```
- If the data is stored as power/angle samples (this is the format that is equivalent to EK60 data), use
    ```
    ds_Sv = ep.calibrate.compute_Sv(ed, waveform_mode="CW", encode_mode="power")
    ```

### Calibration parameters

<!-- For Simrad EK60 and EK80 echosounders, the data file usually already contains the required parameters that can be used directly in calbiration. However, it is not uncommon that custom `cal_params` and `env_params` are used when computing Sv, since parameters stored in the data files may not be the actual values (many of these parameters are manually entered in the instrument settings and remain static there). The AZFP echosounder requires externally supplied  -->
